In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown, HTML
import gseapy as gp
from gseapy import Msigdb
from gseapy import GSEA
from gseapy import dotplot
import warnings
warnings.filterwarnings('ignore')

In [ ]:
adata = sc.read('../Data/adata_with_embeddings.h5ad')

In [ ]:
adata

In [ ]:
# load pam50 and get top 10 genes per subtype as markers
pam50=pd.read_csv('../Data/pam50.tsv',sep='\t',index_col=0)
markers = {}

for subtype in pam50.columns:
    markers[subtype]=pam50.sort_values(by=subtype,ascending=False).index[:10].to_list()

In [ ]:
sc.pp.neighbors(adata, use_rep='X_ae', n_neighbors=7, knn=True, metric='cosine')

In [ ]:
sc.tl.umap(adata, min_dist=.1)#, spread=.5)
sc.tl.leiden(adata,flavor="igraph", n_iterations=2, resolution=.7)



In [ ]:
for k,v in markers.items():
    sc.tl.score_genes(adata, gene_list=v, score_name=k)

In [ ]:
sc.pl.umap(adata, color=list(markers.keys())+['pam50 subtype', 'leiden','nhg'])

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='leiden', method='wilcoxon')
sc.tl.dendrogram(adata, groupby='leiden', use_rep='X_ae')

In [ ]:
from matplotlib.gridspec import GridSpec
clusters_df=sc.get.obs_df(adata, keys=['leiden','pam50 subtype','nhg']+list(markers.keys()))
msig = Msigdb()
gmt = msig.get_gmt(category='h.all', dbver="2025.1.Hs")

def plot_bars(tmp, ax):
    col1_props = tmp['pam50 subtype'].value_counts(normalize=True)
    col2_props = tmp['nhg'].value_counts(normalize=True)
    proportions = pd.concat([col1_props, col2_props], axis=1, keys=['pam50 subtype', 'nhg']).fillna(0)
    proportions.T.plot(
        kind='bar',
        stacked=True,
        colormap='tab20',
        edgecolor='black',
        ax=ax
    )

    ax.legend()
    ax.tick_params(rotation=0)
    ax.set_ylabel('Proportion')
    return ax


def plot_violin(tmp,ax):
    ax=sns.violinplot(tmp[list(markers.keys())],ax=ax)
    ax.tick_params(rotation=45)
    return ax


def pathways(cluster,ax):
    expr = sc.get.rank_genes_groups_df(adata, group=cluster)[['names','scores']]
    expr.columns = ['gene_name', 'score'] 
    pre_res = gp.prerank(
        rnk=expr,  # DataFrame or path to .rnk file
        gene_sets=gmt,  # Or 'KEGG_2021_Human', 'Reactome_2022', etc.
        permutation_num=1000,  # recommended ≥1000
        seed=42,
        processes=4  # parallelization
    )

    ax = dotplot(pre_res.res2d,
             column="FDR q-val",
             cmap=plt.cm.viridis,
             size=4, # adjust dot size
             cutoff=0.25, show_ring=False,ax=ax)
    return ax


def get_genes(cluster):
    df=sc.get.rank_genes_groups_df(adata, group=cluster)

    df = pd.concat([df.head(),df.tail()],axis=0)
    return df


def plot_cluster_features(cluster):
    
    tmp=clusters_df[clusters_df['leiden']==cluster]


    fig = plt.figure(layout="constrained", figsize=(10,10))
    subfigs = fig.subfigures(3,1, wspace=0.07)
    ax0 = subfigs[0].subplots(1, 2)
    ax0[0]=plot_bars(tmp,ax0[0])
    ax0[1]=plot_violin(tmp,ax0[1])

    ax1 = subfigs[1].subplots(1, 3)
    sc.pl.umap(adata,color='leiden',show=False, ax=ax1[0], legend_loc='on data')
    sc.pl.umap(adata,color='pam50 subtype',show=False, ax=ax1[1])
    sc.pl.umap(adata,color='nhg',show=False, ax=ax1[2])

    ax2 = subfigs[2].subplots(1,1)
    ax2=pathways(cluster,ax2)


    display(Markdown(f"## Cluster {cluster}"))
    display(fig)

    # display(HTML(get_genes(cluster).to_html()))
    
    plt.close()


    

    

# Annotate clusters

In [ ]:
markers_dict={}
clusters=adata.obs['leiden'].cat.categories
plot_cluster_features(clusters[0])

In [ ]:
markers_dict[clusters[0]]='Basal-G3_inflammed'

In [ ]:
plot_cluster_features(clusters[1])

In [ ]:
markers_dict[clusters[1]]='LumA-G2_fibrosis'

In [ ]:
plot_cluster_features(clusters[2])

In [ ]:
markers_dict[clusters[2]]='LumA-G2_ER'

In [ ]:
plot_cluster_features(clusters[3])

In [ ]:
markers_dict[clusters[3]]='Normal-G2_fibrosis'

In [ ]:
plot_cluster_features(clusters[4])

In [ ]:
markers_dict[clusters[4]]='LumA-G2_immune-suppressed'

In [ ]:
plot_cluster_features(clusters[5])

In [ ]:
markers_dict[clusters[5]]='Normal-G2_immune-suppressed'

In [ ]:
plot_cluster_features(clusters[6])

In [ ]:
markers_dict[clusters[6]]='LumB-G3_immune-suppressed'

In [ ]:
plot_cluster_features(clusters[7])

In [ ]:
markers_dict[clusters[7]]='Her2-G3_ROS'

In [ ]:
plot_cluster_features(clusters[8])

In [ ]:
markers_dict[clusters[8]]='LumA-G2_immune-suppressed'

In [ ]:
plot_cluster_features(clusters[9])

In [ ]:
markers_dict[clusters[9]]='Her2-G2_ROS'

In [ ]:
plot_cluster_features(clusters[10])

In [ ]:
markers_dict[clusters[10]]='LumA-G2_immune-suppressed'

In [ ]:
adata.obs['clusternames']=adata.obs['leiden'].replace(markers_dict)

In [ ]:
adata.obs['classification']=adata.obs['leiden'].replace(markers_dict).str.split('_').str[0]

In [ ]:
adata.obs['pathway']=adata.obs['leiden'].replace(markers_dict).str.split('_').str[1]

In [ ]:
sc.pl.umap(adata, color=['clusternames','classification','pathway'],ncols=1)

In [ ]:
adata.write('../Data/dataset_annotated.h5ad')